<a href="https://colab.research.google.com/github/QFrankQ/Applied-Numerical-Optimization/blob/main/ADMM_and_Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Collaborative Filtering**#
In this programming assignment, we will implement a matrix completion method that is one approach to collaborative filtering.  \\

**Step 1**: Step 1: Generate data. Recall that in collaborative filtering, you are given a matrix of user ratings for items. Say the matrix is $M \times N$, where $M$ is the number of users and $N$ is the number of items. 

A key observation in collaborative filtering is that users and items can be roughly grouped into a small number of clusters; so the matrix has some intrinsic ``low-rankness'', with a rank $K$ that is much less than $M$ and $N$. Now, set $M=100$,  $N=50$, and $K=5$, and set the random seed to zero. Generate two factor matrices of dimensions $M \times K$ and $K \times N$, respectively, using *np.random.uniform()*. Now multiply them together, you get a matrix $\mathbf{Z}$ of size $M \times N$, but with rank $K$. 

When a user rates an item, they typically give an integer rating (how many stars out of 5, for example). Now, scale the entries of $\mathbf{Z}$ so that they are in the range $[-2,2]$, and then round them to the nearest integer, i.e., each rating is one in $\{-2,-1,0,1,2\}$. 

Now create a new matrix $\mathbf{Y}$; if $\mathbf{Z}$ contains the rating **every** user would give to **every** item, then we use $\mathbf{Y}$ to characterize the actual ratings we observe, since not every user rates every item. Construct $\mathbf{Y}$ by randomly taking out 80\% of the entries in $\mathbf{Z}$; set these entries to a number that represents "missing", e.g., $-$10; they are not observed. The rest of the entries in $\mathbf{Y}$ are equal to those in $\mathbf{Z}$. Denote $\Omega$ as the set of indices (row, column) that corresponds to observed entries in $\mathbf{Y}$. 

In [ ]:
"""
  Add your code here
"""
import numpy as np
np.set_printoptions(suppress=True)
M = 100
N = 50
K = 5
np.random.seed(0)
MK = np.random.uniform(size = (M,K))
KN = np.random.uniform(size = (K,N))
Z = MK @ KN
max_entry = np.amax(Z)
min_entry = np.amin(Z)
diff = max_entry-min_entry
resize = lambda x: -2 + 4*(x-min_entry)/diff
scaled_Z = resize(Z)
Z = np.rint(scaled_Z)
indices = np.random.choice(2, size=(M,N), replace=True, p=[0.8, 0.2])
Y = Z.copy()
Y[indices == 0] = -10
mask = np.where(indices == 1, indices, 0)



*Answer the questions and discuss your findings here*


**Step 2:** Set up our problem. Say we solve the following optimization problem
\begin{align*}
\underset{\mathbf{X}}{\text{minimize}} & \quad \sum_{(i,j) \in \Omega} \frac{1}{2} (X_{i,j} - Y_{i,j})^2 + \tau \| \mathbf{X} \|_* \\
\text{subject to} & \quad \| \mathbf{X} \|_\infty \leq a,
\end{align*} 
where $X_{i,j}$ denotes the $(i,j)$-th entry of $\mathbf{X}$, $\| \cdot \|_*$ denotes the nuclear norm of a matrix. i.e., sum of its singular values, and $\| \cdot \|_\infty$ denotes the infinity norm of a matrix, i.e., the largest absolute value of its entries. $\tau > 0$ and $a > 0$ are constants. The nuclear norm penalty promotes low-rankness of $\mathbf{X}$; the infinity norm constraint makes sure that the entries of $\mathbf{X}$ are bounded. 

Now you see that solving this problem does not seem easy; there are two annoying things, one is the nuclear norm, the other is the infinity norm constraint. Dealing with them together at the same time seems quite hard, since we do not know how to do a proximal operator for both. Instead, we use the idea of ADMM and use the strategy of divide and conquer: 
\begin{align*}
\underset{\mathbf{X},\mathbf{Z}}{\text{minimize}} & \quad \sum_{(i,j) \in \Omega} \frac{1}{2} (X_{i,j} - Y_{i,j})_2^2 + \tau \| \mathbf{Z} \|_* \\
\text{subject to} & \quad \| \mathbf{X} \|_\infty \leq a, \mathbf{X} = \mathbf{Z}.
\end{align*} 
Note: as we discussed in class, you can swap the places where $\mathbf{X}$ and $\mathbf{Z}$ show up in the nuclear norm regularization/infinity norm constraint.  

We now see a (rather) simple solution strategy at our hands using ADMM. Overall, the subproblem for $\mathbf{X}$ can be solved by proximal gradient; the gradient can be derived from the augmented Lagrangian, and the proximal operator is the same as that in the 1-D total variation denoising dual problem. The subproblem for $\mathbf{Z}$ involves a nuclear norm penalty, but the solution is also something you've done before: it is simply the proximal operator in PA4 applied to the vector of singular values. Implement this by taking the singular value decomposition (SVD), put the vector of singular values through the proximal operator, and reconstruct the matrix. \\




**Step 3)**
Implement these components. The main algorithm should have the following format:



In [ ]:
"""
  Add your code here
"""
import math
#objective for the x sub-problem
#g = lambda X, Y, Z, omega, gamma, t: 1/2 * sum([pow((X[row][column] - Y[row][column]),2) for row, column in omega]) + t/2 * pow(np.linalg.norm((X - Z + gamma), ord = 'fro'), 2)

g = lambda X, Y, Z, gamma, t, mask: 1/2 * np.sum(np.square(X-Y)* mask) + t/2 * pow(np.linalg.norm((X - Z + gamma), ord = 'fro'), 2)
#gradient for the x sub-problem
# def gp(X, Y, Z, gamma, t):
#   grad = np.full((M,N), 0)
#   for i in range(M):
#     for j in range(N):
#       grad[i][j] = t * (X[i][j]-Z[i][j]+gamma[i][j])     
#   for (i, j) in omega:
#     grad[i][j] += (X[i][j] - Y[i][j])
#   return grad

gp = lambda X, Y , Z, gamma, t, mask: t * (X - Z + gamma) + (X-Y)* mask

#proximal operator for the x sub-problem
gprox = lambda A, a: np.maximum(np.minimum(A, a), -a)
  
#z sub-problem
#z sub-problem objective
def f(X, Z, gamma, tau, t):
   return 1/2 * pow(np.linalg.norm((X - Z + gamma), ord = 'fro'), 2) + tau/t * np.linalg.norm(Z, ord = 'nuc')

#soft-threshold on singular values of X+gamma
def fprox(A,tau,t):
  u, s, v = np.linalg.svd(A,full_matrices=False)
  sp = np.maximum(s-tau/t,0)
  return np.dot(u,np.dot(np.diag(sp),v))
  
def mc_admm(Y,lam,a,maxit,tol):
    #Y is the input matrix
    #lam and a are the parameters
    #maxit and tol help you terminate the iterations
    t = 2 #stepsize
    it = 0
    X_change = math.inf
    Z_change = math.inf
    X = np.full((M,N), 0)
    Z = np.full((M,N), 0)
    gamma = np.full((M,N), 0)

    #X subproblem
    inner_maxit = 100
    inner_tol = 1e-3
    
    while it <= maxit and (X_change > tol or Z_change > tol) :
      X_inner_change = math.inf
      x_it = 0
      ss = 1/(1+t)
      X_prev = X
      Z_prev = Z
      while x_it <= inner_maxit and X_inner_change > inner_tol:
        A = X - ss * gp(X, Y, Z, gamma, t, mask)
        new_X = gprox(A, a)
        old_obj = g(X, Y, Z, gamma, t, mask)
        X_inner_change = abs(g(new_X, Y, Z, gamma, t, mask) - old_obj)/ abs(old_obj)
        x_it += 1
        X = new_X
      #Z sub-problem
      new_Z = fprox(X + gamma, lam, t)
      Z = new_Z
      gamma = gamma + X - Z
      it += 1

      #print(f"lam: {lam}, it: {it}")
      X_prev_F = np.linalg.norm(X_prev, ord = 'fro')
      Z_prev_F = np.linalg.norm(Z_prev, ord = 'fro')
      X_change = np.linalg.norm((new_X - X_prev), ord = 'fro')
      Z_change = np.linalg.norm((new_Z - Z_prev), ord = 'fro')
      # print(f"{lam} {np.amax(Z)}")
      # print(f"{lam} {np.amin(Z)}")
      # X_prev_F = X_F
      # Z_prev_F = Z_F
      #print(f"X_change {X_change}")
      # X_boo = X_change > tol
      # Z_boo = Z_change > tol
      # print(f"X_change: {X_change}, Z_change: {Z_change}")
      # print(f"X_F: {X_prev_F}, Z_F: {Z_prev_F}")
      # print(f"x_boo: {X_boo}, z_boo: {Z_boo}\n")
    return X

What you have to implement include: variable initialization, solutions to subproblems, and convergence check. To check convergence, for simplicity, you can compare the values of the matrices $\mathbf{X}$ and $\mathbf{Z}$ in consecutive iterations, if the relative change (in terms of Frobenius norm) is small (read: less than tol), you can terminate the iterations. \\


*Answer the questions and discuss your findings here*


**Step 4a)** Run your code. Tweak the algorithm parameters (maxit and tol); for the outer iterations, set them to $1000$ and $1e-6$, and for the inner iterations, set them to $100$ and $1e-3$. A good value for $t$, the ADMM stepsize parameter, is $2$. Calculate a suitable constant stepsize for your proximal gradient algorithm to solve the subproblem for $X$ using Lipschitz continuity. Set $a=2$ since that's how the data is generated.

In [ ]:
"""
  Add your code here
"""
a = 2
maxit = 1000
tol = 1e-6

*Answer the questions and discuss your findings here*

The Lipschitz constant for proximal gradient of the X-subproblem would be t(8 + the maximum entry of gamma) as the difference of X_ij and Z_ij is at most 4 (ratings) and the difference of X_ij and Y_ij is also 4, and gamma_ij is bounded by the greatest entry of gamma)

**Step 4b)** For different values of $\tau \in \{0, 0.3, 1, 2, 3, 4, 5, 10\}$, visualize the singular values of the final solution for $\mathbf{X}$. What is the impact of $\tau$? Explain. Note that the solution will not be perfectly low-rank (numerical reasons), but you can treat the very small ($\sim 1e-6$) singular values as essentially zero.

In [ ]:
"""
  Add your code here
"""
import matplotlib.pyplot as plt

# fig, axs = plt.subplots(8, figsize = (15, 50))
# fig.tight_layout(pad = 5.0)
# i = 0
tau_list = [0, 0.3, 1, 2, 3, 4, 5, 10]

fig, axs = plt.subplots(8, figsize = (15, 30))
i = 0
for lam in tau_list:
    X = mc_admm(Y,lam,a,maxit,tol)
    u, s, v = np.linalg.svd(X)
    axs[i].plot(s, 'o', label = 'singular values for X')
    axs[i].set_title(f'Lambda = {lam}')
    i+= 1
plt.show()




*Answer the questions and discuss your findings here*

$\tau$ forces the recovered matrix to be low rank.


**Step 4c)** Set $\tau = 5$. Visualize the singular values of $\mathbf{X}$ and the maximum absolute value among the entries of $\mathbf{Z}$ for some iterations before convergence (say you converge in 500 iterations, visualize them for iteration 450, 460, 470, 480, 490, 500) and look at the trend. What can you say about the two subproblems interacting with each other? \\

In [ ]:
"""
  Add your code here
"""
def mc_admm_alt(Y,lam,a,maxit,tol):
    #Y is the input matrix
    #lam and a are the parameters
    #maxit and tol help you terminate the iterations
    t = 2 #stepsize
    it = 0
    X_change = math.inf
    Z_change = math.inf
    X = np.full((M,N), 0)
    Z = np.full((M,N), 0)
    gamma = np.full((M,N), 0)


    all_s = []
    all_abs = [] 
    #X subproblem
    inner_maxit = 100
    inner_tol = 1e-3
    
    while it <= maxit and (X_change > tol or Z_change > tol) :
      X_inner_change = math.inf
      x_it = 0
      ss = 1/(1+t)
      X_prev = X
      Z_prev = Z
      while x_it <= inner_maxit and X_inner_change > inner_tol:
        A = X - ss * gp(X, Y, Z, gamma, t, mask)
        new_X = gprox(A, a)
        old_obj = g(X, Y, Z, gamma, t, mask)
        X_inner_change = abs(g(new_X, Y, Z, gamma, t, mask) - old_obj)/ abs(old_obj)
        x_it += 1
        X = new_X
      #Z sub-problem
      new_Z = fprox(X + gamma, lam, t)
      Z = new_Z
      gamma = gamma + X - Z
      it += 1

      X_prev_F = np.linalg.norm(X_prev, ord = 'fro')
      Z_prev_F = np.linalg.norm(Z_prev, ord = 'fro')
      X_change = np.linalg.norm((new_X - X_prev), ord = 'fro')
      Z_change = np.linalg.norm((new_Z - Z_prev), ord = 'fro')
      X_boo = X_change > tol
      Z_boo = Z_change > tol
      # print(f"Xmax: {np.amax(X)}")
      # print(f"Xmin: {np.amin(X)}")
      # print(f"Zmax: {np.amax(Z)}")
      # print(f"Zmin: {np.amin(Z)}")
      u, s, v = np.linalg.svd(X)
      all_s.append(s)
      all_abs.append(max(abs(np.amax(Z)), abs(np.amin(Z))))
    return X, all_s, all_abs
tau = 5
#fig, axs = plt.subplots(1, figsize = (15, 30))
maxit = 10000
tol = 1e-6
X, all_s, all_abs = mc_admm_alt(Y,tau,2,maxit,tol)

fig, axs = plt.subplots(7, figsize = (15, 30))
last_six_abs = []
for i in range(0, 6)[::-1]:
  print(all_abs[-(i*10+1)])
  last_six_abs.append(all_abs[-(i*10+1)])
  axs[i].plot(all_s[-i*10], 'o')
  axs[i].set_title(f"singular value of X at {i*10}th to last iteration")

axs[6].plot(last_six_abs[::-1], 'o')
axs[6].ticklabel_format(useOffset=False)
#axs[6].set_xlim([5, 0])
axs[6].set_title(f"the maximum absolute value in the last 50, 40, 30, 20, 10, 0 iterations")
plt.show()

*Answer the questions and discuss your findings here*

The singular values became low rank and the absolute maximum entries in Z approaches the boundary of entries, which is 2.


**Step 4d)** Set $\tau = 2$. Let's say the columns (movies) corresponds to the bottom 50 movies rated by IMDB users: see the bottom 100 at
[url](https://www.imdb.com/chart/bottom?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=4da9d9a5-d299-43f2-9c53-f0efa18182cd&pf_rd_r=QFNQ592R0V2AX6RPXYJN&pf_rd_s=right-4&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_ql_8). Look at the $7$-th user (the $7$-th row in the matrix). Among the movies they haven't rated, which 3 movies will you predict them to like the most?

In [ ]:
"""
  Add your code here
"""
tau = 2
X = mc_admm(Y,tau,2,maxit,tol)

predict = X + (-10)* mask
top_three = np.argpartition(predict[6],-3)[-3:]
print(top_three)




[35 41 19]


*Answer the questions and discuss your findings here*

The indices are 35, 41, 19. The predicted movies are Turks in Space, 365 days, and Kazaam.

#Add Colab link here: # 

https://colab.research.google.com/drive/1ey2uQEthcdiMgd4MAO-myZqjM7-NOkyA?authuser=1#scrollTo=iqAnBvPK7Ce7